In [1]:
# Data columns (total 29 columns):
#  #   Column                  Non-Null Count  Dtype  
# ---  ------                  --------------  -----  
#  0   ID                      499 non-null    int64  
#  1   Resort                  499 non-null    object 
#  2   Latitude                499 non-null    float64
#  3   Longitude               499 non-null    float64
#  4   Country                 499 non-null    object 
#  5   Continent               499 non-null    object 
#  6   Price                   499 non-null    int64  
#  7   Season                  499 non-null    object 
#  8   Highest point           499 non-null    int64  
#  9   Lowest point            499 non-null    int64  
#  10  Beginner slopes         499 non-null    int64  
#  11  Intermediate slopes     499 non-null    int64  
#  12  Difficult slopes        499 non-null    int64  
#  13  Total slopes            499 non-null    int64  
#  14  Longest run             499 non-null    int64  
#  15  Snow cannons            499 non-null    int64  
#  16  Surface lifts           499 non-null    int64  
#  17  Chair lifts             499 non-null    int64  
#  18  Gondola lifts           499 non-null    int64  
#  19  Total lifts             499 non-null    int64  
# ...
#  27  country_slope_rank      499 non-null    float64
#  28  country_cannon_rank     499 non-null    float64
# dtypes: float64(6), int64(15), object(8)

In [27]:
# import packages

from dash import Dash, dcc, html, dash_table
import dash_bootstrap_components as dbc
from dash.dependencies import Output, Input
from dash.exceptions import PreventUpdate
from dash_bootstrap_templates import load_figure_template

import plotly.express as px
import pandas as pd

# import data and assign a rank to certain metrics

resorts = (
    pd.read_csv("data/resorts.csv", encoding = "ISO-8859-1")
    .assign(
        country_elevation_rank = lambda x: x.groupby("Country", as_index=False)["Highest point"].rank(ascending=False),
        country_price_rank = lambda x: x.groupby("Country", as_index=False)["Price"].rank(ascending=False),
        country_slope_rank = lambda x: x.groupby("Country", as_index=False)["Total slopes"].rank(ascending=False),
        country_cannon_rank = lambda x: x.groupby("Country", as_index=False)["Snow cannons"].rank(ascending=False),
    ))


#resorts['Resort'] = resorts['Resort'].replace('[^\w\s]', ' ', regex=True)

# application

app = Dash(__name__)

app.layout = html.Div([
    html.H2(id="title"),
    dcc.Slider(id = "maxprice",
               min = 0,
               max = 140,
               step = 20,
               value = 80
               ),
    dcc.Checklist(id = "checklist",
                 options = [
                            {'label': 'Summer skiing', 'value': 'Summer skiing'},
                            {'label': 'Night skiing', 'value': 'Nightskiing'},
                            {'label': 'Snow parks', 'value': 'Snowparks'}
                            ],
                 value = []),
    dcc.Graph(id = "map")
])

@app.callback(
    Output("title", "children"),
    Output("map", "figure"),
    Input("maxprice", "value"),
    Input("checklist", "value")
)

def update_dash(pricelimit, features):
    
    df_filtered = resorts.query("Price <= @pricelimit")

    title = f"Total slope density of ski resorts under ${pricelimit}"

    if len(features) == 1:
        if "Nightskiing" in features:
            df_filtered = df_filtered.query('Nightskiing == "Yes"')
        elif "Summer skiing" in features:
            df_filtered = df_filtered.query('`Summer skiing` == "Yes"')
        else:
            df_filtered = df_filtered.query('Snowparks == "Yes"')
    elif len(features) == 2:
        if "Nightskiing" in features and "Summer skiing" in features :
            df_filtered = df_filtered.query("`Summer skiing` == 'Yes' and Nightskiing == 'Yes'")
        if "Nightskiing" in features and "Snowparks" in features :
            df_filtered = df_filtered.query("Snowparks == 'Yes' and Nightskiing == 'Yes'")
        if "Summer skiing" in features and "Snowparks" in features :
            df_filtered = df_filtered.query("`Summer skiing` == 'Yes' and Snowparks == 'Yes'")
    elif len(features) == 3:
            df_filtered = df_filtered.query("`Summer skiing` == 'Yes' and Nightskiing == 'Yes' and Snowparks == 'Yes'")
    else:
        df_filtered = df_filtered

    figure = px.density_mapbox(
        df_filtered,
        lat = "Latitude",
        lon = "Longitude",
        z = "Total slopes",
        mapbox_style = "open-street-map",
        zoom = 2.5,
        center = {"lat": 52.5, "lon": 15},
        height = 600,
        width = 800,
        radius = 15,
        hover_name="Resort")

    return title, figure

if __name__ == "__main__":
    app.run_server(port = 8051, debug = True)

In [149]:
features = ["Summer skiing", 'Nightskiing', "Snowparks"]

df = resorts

if len(features) == 1:
    if "Nightskiing" in features:
        df = df.query('Nightskiing == "Yes"')
    elif "Summer skiing" in features:
        df = df.query('`Summer skiing` == "Yes"')
    else:
        df = df.query('Snowparks == "Yes"')
elif len(features) == 2:
    if "Nightskiing" in features and "Summer skiing" in features :
        df = df.query("`Summer skiing` == 'Yes' and Nightskiing == 'Yes'")
    if "Nightskiing" in features and "Snowparks" in features :
        df = df.query("Snowparks == 'Yes' and Nightskiing == 'Yes'")
    if "Summer skiing" in features and "Snowparks" in features :
        df = df.query("`Summer skiing` == 'Yes' and Snowparks == 'Yes'")
elif len(features) == 3:
        df = df.query("`Summer skiing` == 'Yes' and Nightskiing == 'Yes' and Snowparks == 'Yes'")
else:
    df = resorts


9
